for Binary classification - we are using Logistic regression

importing dependencies

In [28]:
import numpy as np
import pandas as pd # to put it in structured table dataframe
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

data collection & processing

In [29]:
#loading the csv data to a pandas dataframe

In [30]:
heart_data=pd.read_csv('../dataset/heart.csv')

In [31]:
heart_data.head()
heart_data.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


In [32]:
heart_data.shape

(303, 14)

getting some info abt data

In [33]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


checking for missing values

In [34]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

statistical measures for our data,    25% data out of total in age are less than 47.5

In [35]:
heart_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


how many people has heart disease and not, distribution of target variable

In [36]:
heart_data['target'].value_counts()

target
1    165
0    138
Name: count, dtype: int64

split features and target

In [37]:
x=heart_data.drop(columns='target',axis=1)
y=heart_data['target']

In [38]:
x

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


In [39]:
y

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64

split our data into train , test

In [40]:
trainx,testx,trainy,testy=train_test_split(x,y, test_size=0.2, stratify=y,  random_state=63)

In [41]:
print(x.shape,trainx.shape,testx.shape)

(303, 13) (242, 13) (61, 13)


Now we train our model

In [42]:
# tomek = TomekLinks(sampling_strategy='auto')
# smote = SMOTE(random_state=32)
# xtrain, ytrain = smote.fit_resample(trainx, trainy) #  without tomek rec=68, with-72
# xtrain, ytrain = tomek.fit_resample(trainx, trainy)

In [43]:
model = LogisticRegression(
    C=0.1,               # Regularization strength
    class_weight='balanced',  # Adjust class weights to handle class imbalance
    max_iter=100,        # Number of iterations for solver to converge
    solver='liblinear'   # Solver to use (liblinear is good for small datasets)
)

now training with train data

In [44]:
model.fit(trainx,trainy)

LogisticRegression(C=0.1, class_weight='balanced', solver='liblinear')

model evaluation , use acc score as our evalution metric

In [45]:
#accuracy on training data
xtrain_pred=model.predict(trainx)
training_data_accuracy=accuracy_score(xtrain_pred, trainy)

In [46]:
print("Accuracy on training data: ",training_data_accuracy)

Accuracy on training data:  0.859504132231405


In [47]:
#on test data
xtest_pred=model.predict(testx)
test_data_accuracy=accuracy_score(xtest_pred, testy)

In [48]:
print("Accuracy on test data: ",test_data_accuracy)

Accuracy on test data:  0.8360655737704918


In [49]:
cm=confusion_matrix(testy,xtest_pred)
print("Confusion Matrix: ",cm)

Confusion Matrix:  [[21  7]
 [ 3 30]]


In [50]:
pre=precision_score(testy,xtest_pred)
print("Precision: ",pre)

Precision:  0.8108108108108109


In [51]:
re=recall_score(testy,xtest_pred)
print("Recall: ",re)

Recall:  0.9090909090909091


build a predicitive system.

In [60]:
inputdata=(63,0,0,150,407,0,0,154,0,4,1,3,3)
#inputdata=(62,0,2,130,263,0,1,97,0,1.2,1,1,3)

inpdata_asnumpyar=np.asarray(inputdata)

#reshape numpy arr , to tell find prediction for only 1 instance
inpdata_reshaped=inpdata_asnumpyar.reshape(1,-1)

prediction=model.predict(inpdata_reshaped)
print(prediction)

if(prediction[0]==0):
  print("person does not have heart disease")
else:
  print("person has heart disease")

[0]
person does not have heart disease


C:\Users\ritti\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [26]:
import joblib

joblib.dump(model,"../models/heart_model.pkl")

['../models/heart_model.pkl']